# Korrelation zwischen den Handelsdaten Import, Export, BIP und den Militärausgaben mit Spearman R prüfen

## Benötigte Bibliotheken

In [33]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import pycountry
import plotly.express as px

## Länder filtern
#### In den Datensätzen befinden sich Spalten die kein Länder darstellen, sondern zum Beispiel einen ganzen Kontinent, aber wir möchten für die Spearman Korrelation nur einzelne Länder untersuchen.
#### Wir sortieren diese mittels einer Keyword List raus
#### Es gibt eine zweite Keyword Liste für Länder die dadurch aussortiert werden würden, um dafür zu sorgen, dass diese weiterhin im Datensatz bleiben

In [34]:
non_country_keywords = [
    "Africa", "Europe", "America", "Asia", "Oceania", "Arab", "World",
    "High income", "Low income", "Middle income", "Euro area", "OECD", "IDA",
    "IBRD", "G7", "G20", "Least developed", "Landlocked", "Small states", "Sub-Saharan",
    "Heavily indebted", "demographic dividend", "middle income", "Other"
]

allowlist = {
    "Egypt, Arab Rep.", "Saudi Arabia", "South Africa",
    "Syrian Arab Republic", "United Arab Emirates", "Central African Republic"
}

In [35]:
def is_region_or_group(name):
    return (
        any(keyword in str(name) for keyword in non_country_keywords)
        and name not in allowlist
    )

#### Länder ohne Daten werden auch rausgefiltert.

In [36]:
def clean_df(df):
    df = df[~df["Country Name"].apply(is_region_or_group)]
    mask = (df.iloc[:, 3:].fillna(0) != 0).any(axis=1)
    df = df[mask].reset_index(drop=True)
    return df

#### Überprüfen ob nach dem Bereinigen des Datensatzes die übrigen Länder in beiden Datensätzen vorkommen, falls nicht werden sie aussortiert.

In [37]:
def filter_for_same_columns(df1, df2):
    set1 = set(df1["Country Name"])
    set2 = set(df2["Country Name"])

    common = set1 & set2

    df1_clean = df1[df1["Country Name"].isin(common)].reset_index(drop=True)
    df2_clean = df2[df2["Country Name"].isin(common)].reset_index(drop=True)
    return df1_clean, df2_clean

# Wichtige Funktionen zur Berechnung und Visualisierung der Korrelation

## Spearman R Korrelation und Signifikanz berechnen
#### Es wird für jedes Land die Korrelation mit Spearman r berechnet zwischen einem Handelsdatensatz und dem Datensatz der Militärausgaben. Alle Werte werden in nummerische umgewandelt. Dann wird noch überprüft für welche Jahre es bei beiden Handelsdaten Werte gibt und diese werden dann für die Korrelation verwendet, sofern es midestens 3 Wertepaare gibt, sonst wird keine Korrelation berechnet. Zum Schluss werden die Daten in einer Tabelle gespeichert mit dem Land, der zugehörigen Anzahl an Jahre die füe die Berechnung verwendet wurden, dem Werte der Spearman R Korrelation, sowie die Signifikanz des Ergebnisses.

In [38]:
def calc_spearman(df_trade, df_military):
    df = df_trade.merge(df_military, on="Country Name", suffixes=("_trade", "_mil"))

    years = [c for c in df_trade.columns if c.isdigit() and c in df_military.columns]

    results = []
    for _, row in df.iterrows():
        country = row["Country Name"]

        x = pd.to_numeric(row[[f"{y}_trade" for y in years]], errors="coerce").to_numpy()
        y = pd.to_numeric(row[[f"{y}_mil" for y in years]], errors="coerce").to_numpy()
        
        mask = (~np.isnan(x)) & (x != 0) & (~np.isnan(y)) & (y != 0)
        x_valid = x[mask]
        y_valid = y[mask]
        
        if len(x_valid) < 3:
            continue

        rho, p_s = spearmanr(x_valid, y_valid)
        
        results.append({
            "Country Name": country,
            "n Jahre": len(x_valid),
            "Spearman ρ": round(rho, 3),
            "p-Spearman": round(p_s, 3),
        })

    df_country_corr = pd.DataFrame(results) \
        .sort_values("Spearman ρ", ascending=False) \
        .reset_index(drop=True)
    return df_country_corr

## Interaktive Weltkarte erzeugen
#### Mit dieser Funktion wird eine interaktive Weltkart erstellt. Hierzu werden alle Korrelaationsdaten aus einem neu erstellten Datensatz gezogen und dann für die Länder der jeweilige Korrelationswert angegeben und dementsprechend eingefärbt.

In [39]:
def create_world_map(file, trade):
    df_corr = pd.read_csv(file)

    def iso3(name):
        try:
            return pycountry.countries.lookup(name).alpha_3
        except LookupError:
            return None

    df_corr['iso_a3'] = df_corr['Country Name'].apply(iso3)

    df_map = df_corr.dropna(subset=['iso_a3'])

    fig = px.choropleth(
        df_map,
        locations='iso_a3',
        color='Spearman ρ',
        hover_name='Country Name',
        color_continuous_scale='OrRd',
        projection='natural earth',
        title=f'Interaktive Spearman-ρ Karte: {trade} vs. Militär'
    )

    fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
    fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
    fig.show()
    return df_corr

# Spearman R Korrelation Importe - Militärausgaben
#### Wir möchten hier prüfen ob es einen Zusammenhang zwischen den Importdaten eines Landes und den Militärausgaben gibt. H1: Korreliert das Militärbudget eines Landes mit den Importdaten des Landes?


In [40]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military = clean_df(df_military)

In [41]:
df_import = pd.read_csv("data/imports.csv", sep=',', skiprows=4)
df_import = df_import.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_import = clean_df(df_import)

In [42]:
df_import, df_military = filter_for_same_columns(df_import, df_military)


In [43]:
df_country_corr = calc_spearman(df_import, df_military)
print(df_country_corr)
df_country_corr.to_csv("results/import_correlations.csv", index=False)

               Country Name  n Jahre  Spearman ρ  p-Spearman
0               Afghanistan        8       1.000       0.000
1                     Nepal       54       0.992       0.000
2                     China       35       0.992       0.000
3                 Singapore       54       0.991       0.000
4                Luxembourg       54       0.990       0.000
..                      ...      ...         ...         ...
153                  Guyana       30      -0.117       0.539
154  Bosnia and Herzegovina       22      -0.240       0.282
155            Turkmenistan        6      -0.314       0.544
156                 Lao PDR       22      -0.319       0.148
157             South Sudan        8      -0.595       0.120

[158 rows x 4 columns]


In [44]:
df_corr_import = create_world_map('results/import_correlations.csv', 'Import')

#### Auf der Weltkarte gibt es einige grau gefärbte Länder, dies sind Staaten für die keine Korrelation berechnet werden konnte. Man kann erkennen, dass es zwischen den Importdaten und den Militärausgaben vieler Länder eine hohe Korrelation gibt.

In [45]:
nepal = df_corr_import.loc[df_corr_import["Country Name"] == "Nepal"]
print("Nepal Signifikanz und Korrelation")
print(nepal['p-Spearman'].iloc[0])
print(nepal['Spearman ρ'].iloc[0])

china = df_corr_import.loc[df_corr_import["Country Name"] == "China"]
print("China Signifikanz und Korrelation")
print(china['p-Spearman'].iloc[0])
print(china['Spearman ρ'].iloc[0])

singapore = df_corr_import.loc[df_corr_import["Country Name"] == "Singapore"]
print("Singapore Signifikanz und Korrelation")
print(singapore['p-Spearman'].iloc[0])
print(singapore['Spearman ρ'].iloc[0])

Nepal Signifikanz und Korrelation
0.0
0.992
China Signifikanz und Korrelation
0.0
0.992
Singapore Signifikanz und Korrelation
0.0
0.991


#### Für Nepal, China und Singapore z.B. gibt es eine sehr hohe Signifikanz. Diese Länder weisen eine nahezu perfekte positive Korrelation auf.

In [46]:
zimbabwe = df_corr_import.loc[df_corr_import["Country Name"] == "Zimbabwe"]
print("Zimbabwe Signifikanz und Korrelation")
print(zimbabwe['p-Spearman'].iloc[0])
print(zimbabwe['Spearman ρ'].iloc[0])

guyana = df_corr_import.loc[df_corr_import["Country Name"] == "Guyana"]
print("Guyana Signifikanz und Korrelation")
print(guyana['p-Spearman'].iloc[0])
print(guyana['Spearman ρ'].iloc[0])

libya = df_corr_import.loc[df_corr_import["Country Name"] == "Libya"]
print("Lybien Signifikanz und Korrelation")
print(libya['p-Spearman'].iloc[0])
print(libya['Spearman ρ'].iloc[0])

Zimbabwe Signifikanz und Korrelation
0.988
0.002
Guyana Signifikanz und Korrelation
0.539
-0.117
Lybien Signifikanz und Korrelation
0.334
0.268


#### Es gibt auch einige wenige Beispiele, wo dies nicht der Fall ist z.B. Zimbabwe hat kein bisschen Signifikanz und überhaupt keine Korrelation. Guyana weißt auch keine Signifikanz und hat eine leichte negative Korrelation. Auch für Lybien ist dies der Fall hier sind keine Signifikanz vorzuweisen und nur eine leichte Korrelation von 0.268.

In [47]:
print(df_corr_import['p-Spearman'].mean())
print(df_corr_import['Spearman ρ'].mean())

0.02718987341772152
0.822632911392405


#### Insgesamt kann man aber anhand der durchschnitlichen Signifikanz der Länder, die deutlich unter 5% ist, kann man sehen, dass die Hypothese "H1: Korreliert das Militärbudget eines Landes mit den Importdaten des Landes?" signifikant ist. Und durch die durchschnittliche Korrelation von fast 0.82 ein starker Zusammenhang zwischen dem Import und den Militärausgaben eines Landes festgestellt werden kann.

# Spearman R Korrelation Export-Militärausgaben
#### H2: Korreliert das Militärbudget eines Landes mit den Exportdaten des Landes?

In [48]:
df_export = pd.read_csv("data/exports.csv", sep=',', skiprows=4)
df_export = df_export.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_export = clean_df(df_export)

In [49]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military = clean_df(df_military)

In [50]:
df_export, df_military = filter_for_same_columns(df_export, df_military)

In [51]:
df_country_corr = calc_spearman(df_export, df_military)
print(df_country_corr)
df_country_corr.to_csv("results/export_correlations.csv", index=False)

               Country Name  n Jahre  Spearman ρ  p-Spearman
0                 Singapore       54       0.992       0.000
1                     China       35       0.991       0.000
2                 Australia       64       0.991       0.000
3                Bangladesh       51       0.991       0.000
4                    Mexico       64       0.991       0.000
..                      ...      ...         ...         ...
152                  Guyana       30      -0.160       0.398
153             South Sudan        8      -0.262       0.531
154                 Lao PDR       22      -0.412       0.057
155  Bosnia and Herzegovina       22      -0.486       0.022
156                 Eritrea       11      -0.755       0.007

[157 rows x 4 columns]


In [52]:
df_corr_export = create_world_map('results/export_correlations.csv', 'Export')

#### Auf der Weltkarte gibt es einige grau gefärbte Länder, dies sind Staaten für die keine Korrelation berechnet werden konnte. Man kann erkennen, dass es zwischen dem Export und den Militärausgaben vieler Länder eine hohe Korrelation gibt.

In [53]:
australia = df_corr_export.loc[df_corr_export["Country Name"] == "Australia"]
print("Australien Signifikanz und Korrelation")
print(australia['p-Spearman'].iloc[0])
print(australia['Spearman ρ'].iloc[0])

mexico = df_corr_export.loc[df_corr_export["Country Name"] == "Mexico"]
print("Mexiko Signifikanz und Korrelation")
print(mexico['p-Spearman'].iloc[0])
print(mexico['Spearman ρ'].iloc[0])

poland = df_corr_export.loc[df_corr_export["Country Name"] == "Poland"]
print("Polen Signifikanz und Korrelation")
print(poland['p-Spearman'].iloc[0])
print(poland['Spearman ρ'].iloc[0])

Australien Signifikanz und Korrelation
0.0
0.991
Mexiko Signifikanz und Korrelation
0.0
0.991
Polen Signifikanz und Korrelation
0.0
0.985


#### Besonders hohe Korrelation haben hier z.B. Australien, Mexiko und Polen mit Werten um die 0.99. Zudem weißen sie eine hohe Signifikanz auf.

In [54]:
croatia = df_corr_export.loc[df_corr_export["Country Name"] == "Croatia"]
print("Kroatien Signifikanz und Korrelation")
print(croatia['p-Spearman'].iloc[0])
print(croatia['Spearman ρ'].iloc[0])

turkmenistan = df_corr_export.loc[df_corr_export["Country Name"] == "Turkmenistan"]
print("Turkmenistan Signifikanz und Korrelation")
print(turkmenistan['p-Spearman'].iloc[0])
print(turkmenistan['Spearman ρ'].iloc[0])

southsudan = df_corr_export.loc[df_corr_export["Country Name"] == "South Sudan"]
print("Süd Sudan Signifikanz und Korrelation")
print(southsudan['p-Spearman'].iloc[0])
print(southsudan['Spearman ρ'].iloc[0])

Kroatien Signifikanz und Korrelation
0.571
0.11
Turkmenistan Signifikanz und Korrelation
0.787
-0.143
Süd Sudan Signifikanz und Korrelation
0.531
-0.262


#### Es gibt aber auch ein paar Ausnahmen z.B. Kroatien weißt keine Signifikanz auf und hat auch nur eine nicht nennenswerte Korrelation von 0.11. Genauso Turkmenistan mit einer negativen Korrelation von -0.143. Ebenso gibt es beim Süd Sudan keine Signifikanz.

In [55]:
print(df_corr_export['p-Spearman'].mean())
print(df_corr_export['Spearman ρ'].mean())

0.02942675159235669
0.7925668789808916


#### An der durchschnitlichen Signifikanz der Länder, die deutlich unter 5% ist, kann aber man sehen, dass die Hypothese "H2: Korreliert das Militärbudget eines Landes mit den Exportdaten des Landes?" in den meisten Fällen signifikant ist. Und durch die durchschnittliche Korrelation von fast 0.8 ein starker Zusammenhang zwischen dem Export und dem Militär eines Landes festgestellt werden kann.

# Spearman R Korrelation BIP - Militärausgaben
#### In diesem Schritt möchten wir prüfen, ob es einen Zusammenhang zwischen dem BIP eines Landes und seinen Militärausgaben gibt. H3: Korreliert das Militärbudget eines Landes mit dem Bruttoinlandsprodukt des Landes?

In [56]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military = clean_df(df_military)

In [57]:
df_gdp = pd.read_csv("data/gdp.csv", sep=',', skiprows=4)
df_gdp = df_gdp.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_gdp = clean_df(df_gdp)

#### Wir nehmen wieder Länder raus die nur in einem Datensatz vorkommen.

In [58]:
df_gdp, df_military = filter_for_same_columns(df_gdp, df_military)


#### Jetzt berechnen wir die Korrelation zwsichen dem BIP und den Militörausgaben der einzelnen Länder

In [59]:
df_country_corr = calc_spearman(df_gdp, df_military)
print(df_country_corr)
df_country_corr.to_csv("results/gdp_correlations.csv", index=False)

               Country Name  n Jahre  Spearman ρ  p-Spearman
0                Uzbekistan       10       1.000       0.000
1               Korea, Rep.       64       0.997       0.000
2                   Estonia       31       0.996       0.000
3                     India       64       0.995       0.000
4                     China       35       0.995       0.000
..                      ...      ...         ...         ...
160                 Croatia       32       0.339       0.057
161            Turkmenistan        6       0.143       0.787
162             South Sudan        8      -0.167       0.693
163  Bosnia and Herzegovina       22      -0.203       0.366
164                 Lao PDR       22      -0.309       0.162

[165 rows x 4 columns]


#### Jetzt erstellen wir wieder eine interaktive Weltkarte, zur Veranschaulichung des Zusammenhangs von BIP und Militärausgaben

In [60]:
df_corr_gdp =create_world_map('results/gdp_correlations.csv', 'BIP')

#### Auch beim BIP gibt es einige Länder die rausfallen wegen unzureichender Daten. Trotzdem lässte sich auf der Weltkarte erkennen, dass es auch zwischen BIP und Militärausgaben einen hohen Zusammenhang gibt.

In [61]:
estonia = df_corr_gdp.loc[df_corr_gdp["Country Name"] == "Estonia"]
print("Estland Signifikanz und Korrelation")
print("p: ", estonia['p-Spearman'].iloc[0])
print("ρ (rho): ", estonia['Spearman ρ'].iloc[0])

india = df_corr_gdp.loc[df_corr_gdp["Country Name"] == "India"]
print("Indien Signifikanz und Korrelation")
print("p: ", india['p-Spearman'].iloc[0])
print("ρ (rho): ", india['Spearman ρ'].iloc[0])

brazil = df_corr_gdp.loc[df_corr_gdp["Country Name"] == "Brazil"]
print("Brasilien Signifikanz und Korrelation")
print("p: ", brazil['p-Spearman'].iloc[0])
print("ρ (rho): ", brazil['Spearman ρ'].iloc[0])


Estland Signifikanz und Korrelation
p:  0.0
ρ (rho):  0.996
Indien Signifikanz und Korrelation
p:  0.0
ρ (rho):  0.995
Brasilien Signifikanz und Korrelation
p:  0.0
ρ (rho):  0.991


#### Beispiele mit hoher Signifikanz sind hier Estland, Indien und Brasilien, diese Länder weisen alle einen hohen Zusammenhang zwischen dem ihrem BIP und Militärausgaben auf.

In [62]:
panama = df_corr_gdp.loc[df_corr_gdp["Country Name"] == "Panama"]
print("Panama Signifikanz und Korrelation")
print("p: ", panama['p-Spearman'].iloc[0])
print("ρ (rho): ", panama['Spearman ρ'].iloc[0])

bosnia = df_corr_gdp.loc[df_corr_gdp["Country Name"] == "Bosnia and Herzegovina"]
print("Bosnien Signifikanz und Korrelation")
print("p: ", bosnia['p-Spearman'].iloc[0])
print("ρ (rho): ", bosnia['Spearman ρ'].iloc[0])

turkmenistan = df_corr_gdp.loc[df_corr_gdp["Country Name"] == "Turkmenistan"]
print("Turkmenistan Signifikanz und Korrelation")
print("p: ", turkmenistan['p-Spearman'].iloc[0])
print("ρ (rho): ", turkmenistan['Spearman ρ'].iloc[0])

Panama Signifikanz und Korrelation
p:  0.144
ρ (rho):  0.429
Bosnien Signifikanz und Korrelation
p:  0.366
ρ (rho):  -0.203
Turkmenistan Signifikanz und Korrelation
p:  0.787
ρ (rho):  0.143


#### Es gibt einige wenige Beispiele mit keiner vorzuweisender Siginfikanz, darunter Panama, Bosnien und wieder Turkmenistan. Panama hat eine nicht allzu hohe Korrelation mit 0.429, bei Bosnien ist nur eine geringe negative Korrelation von -0.2 festzustellen. Und bei Turkmenistan sind es nur 0.143.

In [63]:
print(df_corr_gdp['p-Spearman'].mean())
print(df_corr_gdp['Spearman ρ'].mean())

0.015490909090909092
0.855812121212121


#### Dennoch erkennt man an der durchschnitlichen Signifikanz der Länder, die deutlich sogar unter 2% liegt, kann man sehen, dass die Hypothese "H3: Korreliert das Militärbudget eines Landes mit dem Bruttoinlandsprodukt des Landes?" für die meisten Länder des Datensatzes signifikant ist. Und durch die durchschnittliche Korrelation von über 0.85 gibt es sogar einen noch stärkeren Zusammenhang wie beim Export zwischen dem BIP und dem Militär eines Landes.